In [1]:
import sys, os
from os import path
sys.path.append('/root/bdrisk/risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *
from risk_models import _name_BD_RISK_RESULT_STOCK_ST1, _name_BD_RISK_RESULT_STOCK_ST2, _name_BD_RISK_RESULT_STOCK_ST3, _name_BD_RISK_RESULT_STOCK_ST4, _name_BD_RISK_RESULT_STOCK_ST5


class ScoreSt(object):
    def __init__(self, child_task_id, org_code, params):
        self.child_task_id = child_task_id
        Risk_logger(child_task_id = self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        self.weights = json.loads(params)['weights']

    def score_stock(self):
        # 读取企业库存模块的结果表数据
        ST1_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST1, self.org_code), database = 'dbods')
        ST2_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST2, self.org_code), database = 'dbods')
        ST3_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST3, self.org_code), database = 'dbods')
        ST4_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST4, self.org_code), database = 'dbods')
        ST5_RESULT = Read_Oracle().read_oracle(sql= """ select * from {} where org_code = '{}' and iscurrent = 1 """.format(_name_BD_RISK_RESULT_STOCK_ST5, self.org_code), database = 'dbods')
        RESULT_LIST = [ST1_RESULT, ST2_RESULT, ST3_RESULT, ST4_RESULT, ST5_RESULT]
        
        # 计算各个子模块分数（没有数据的初始化为0分）
        for i in range(5):
            if RESULT_LIST[i] is None:
                RESULT_LIST[i] = 0
            else:
                RESULT_LIST[i] = 100 + RESULT_LIST[i]['SCORE'].sum()
        
        # 设置子模块权重(通过json参数获取)
        RATIO = self.weights
        STOCK_SCORE = 0
        
        # 对所有子模型分数加权计算总分
        for i in range(5):
            STOCK_SCORE = STOCK_SCORE + RESULT_LIST[i] * RATIO[i]
        
        # 整理分数表并写入数据库
        STOCK_RESULT = pd.DataFrame(data=[[0, self.org_code, 'STOCK', STOCK_SCORE]], columns=['ID', 'ORG_CODE', 'MODEL_CODE', 'SCORE'])
        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        STOCK_RESULT['CHECK_TIME'] = datetime.datetime.strptime(now, "%Y-%m-%d %H:%M:%S")
        Write_Oracle().write_oracle('BD_RISK_CORP_SCORE_DISPLAY',STOCK_RESULT, org_code = self.org_code, alarm = ['STOCK',''])

    def run_score_stock(self):
        try:
            self.score_stock()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id = self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    # child_task_id = sys.argv[1]
    child_task_id = '0003_0001'
    org_code, params, base_time = read_log_table(child_task_id)
    ScoreSt(child_task_id, org_code, params).run_score_stock()



2021-06-07 17:12:14.107 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 4.356s
2021-06-07 17:12:16.575 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.429s
2021-06-07 17:12:17.759 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 1.180s
2021-06-07 17:12:17.887 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.126s
2021-06-07 17:12:18.084 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.194s
2021-06-07 17:12:18.300 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.213s
2021-06-07 17:12:18.428 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 -